In [1]:
!pip install -qq pytabkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
import pandas as pd, numpy as np

train = pd.read_csv('/kaggle/input/playground-series-s5e10/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e10/test.csv')
orig = pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv')
print('Train Shape:', train.shape)
print('Test Shape:', test.shape)
print('Orig Shape:', orig.shape)

train.head(3)

Train Shape: (517754, 14)
Test Shape: (172585, 13)
Orig Shape: (100000, 13)


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30


In [4]:
TARGET = 'accident_risk'
BASE = [col for col in train.columns if col not in ['id', TARGET]]
CATS = ['road_type', 'lighting', 'weather', 'road_signs_present', 'public_road', 'time_of_day', 'holiday', 'school_season']

print(f'{len(BASE)} Base Features:{BASE}')

12 Base Features:['road_type', 'num_lanes', 'curvature', 'speed_limit', 'lighting', 'weather', 'road_signs_present', 'public_road', 'time_of_day', 'holiday', 'school_season', 'num_reported_accidents']


In [5]:
ORIG = []

for col in BASE:
    tmp = orig.groupby(col)[TARGET].mean()
    new_col_name = f"orig_{col}"
    tmp.name = new_col_name
    train = train.merge(tmp, on=col, how='left')
    test = test.merge(tmp, on=col, how='left')
    ORIG.append(new_col_name)

print(len(ORIG), 'Orig Features Created!!')

12 Orig Features Created!!


In [6]:
META = []

for df in [train, test, orig]:
    base_risk = (
        0.3 * df["curvature"] + 
        0.2 * (df["lighting"] == "night").astype(int) + 
        0.1 * (df["weather"] != "clear").astype(int) + 
        0.2 * (df["speed_limit"] >= 60).astype(int) + 
        0.1 * (np.array(df["num_reported_accidents"]) > 2).astype(int)
    )
    df['Meta'] = base_risk

META.append('Meta')

In [7]:
train['orig_curvature'] = train['orig_curvature'].fillna(orig[TARGET].mean())
test['orig_curvature'] = test['orig_curvature'].fillna(orig[TARGET].mean())

In [8]:
FEATURES = BASE + ORIG + META
print(len(FEATURES), 'Features.')

25 Features.


In [9]:
X = train[FEATURES]
y = train[TARGET]
X_test = test[FEATURES]

In [10]:
from sklearn.model_selection import KFold

N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

In [11]:
# from pytabkit import RealMLP_HPO_Regressor

# model = RealMLP_HPO_Regressor(n_cv=8,
#                               n_epochs=64,
#                               hpo_space_name='tabarena', 
#                               use_caruana_ensembling=False, 
#                               n_hyperopt_steps=30, 
#                               verbosity=2)
# model.fit(X, y, cat_col_names=CATS)

In [12]:
from pytabkit import RealMLP_TD_Regressor

from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler

In [13]:
params = {'n_epochs': 64,
          'verbosity': 2,
          'hidden_sizes': 'rectangular',
          'max_one_hot_cat_size': 9,
          'embedding_size': 8, 
          'weight_param': 'ntk', 
          'weight_init_mode': 'std',
          'bias_init_mode': 'he+5',
          'bias_lr_factor': 0.1,
          'act': 'mish',
          'use_parametric_act': True, 
          'act_lr_factor': 0.1,
          'wd': 0.021237565639937887,
          'wd_sched': 'flat_cos',
          'bias_wd_factor': 0.0,
          'block_str': 'w-b-a-d',
          'p_drop': 0.11884235947908856,
          'p_drop_sched': 'flat_cos',
          'add_front_scale': True,
          'scale_lr_factor': 2.199981685025475,
          'tfms': ['one_hot', 'median_center', 'robust_scale', 'smooth_clip', 'embedding'],
          'num_emb_type': 'pbld', 
          'plr_sigma': 29.684675895967988,
          'plr_hidden_1': 16,
          'plr_hidden_2': 64,
          'plr_lr_factor': 0.21570704623708598,
          'clamp_output': True,
          'normalize_output': True, 
          'lr': 0.02722614375554257,
          'lr_sched': 'coslog4',
          'opt': 'adam',
          'sq_mom': 0.9663775995449818,
          'n_hidden_layers': 4,
          'hidden_width': 512, 
          'first_layer_lr_factor': 0.6988631671008149,
          'ls_eps_sched': 'coslog4',
          'ls_eps': 0.011689238765538871,
          'use_ls': True,
          'use_early_stopping': True,
          'early_stopping_multiplicative_patience': 3,
          'early_stopping_additive_patience': 40,
         }

In [14]:
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(test))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f'--- Fold {fold+1}/{N_SPLITS} ---')
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = RealMLP_TD_Regressor(**params)
    
    model.fit(X_train, y_train, X_val, y_val, cat_col_names=CATS)
    
    oof_preds[val_idx] = model.predict(X_val)
    test_preds += model.predict(X_test)

    print(f"Fold {fold+1} RMSE: {root_mean_squared_error(y_val, oof_preds[val_idx]):.5f}")

test_preds /= N_SPLITS

print(f"Overall OOF RMSE: {root_mean_squared_error(y, oof_preds):.5f}")

--- Fold 1/5 ---
Columns classified as continuous: ['num_lanes', 'curvature', 'speed_limit', 'num_reported_accidents', 'orig_road_type', 'orig_num_lanes', 'orig_curvature', 'orig_speed_limit', 'orig_lighting', 'orig_weather', 'orig_road_signs_present', 'orig_public_road', 'orig_time_of_day', 'orig_holiday', 'orig_school_season', 'orig_num_reported_accidents', 'Meta']
Columns classified as categorical: ['road_type', 'lighting', 'weather', 'road_signs_present', 'public_road', 'time_of_day', 'holiday', 'school_season']
Epoch 1/64: val rmse = 0.058039
Epoch 2/64: val rmse = 0.057805
Epoch 3/64: val rmse = 0.057671
Epoch 4/64: val rmse = 0.056414
Epoch 5/64: val rmse = 0.056598
Epoch 6/64: val rmse = 0.057286
Epoch 7/64: val rmse = 0.058162
Epoch 8/64: val rmse = 0.059228
Epoch 9/64: val rmse = 0.057221
Epoch 10/64: val rmse = 0.057290
Epoch 11/64: val rmse = 0.056924
Epoch 12/64: val rmse = 0.056270
Epoch 13/64: val rmse = 0.056211
Epoch 14/64: val rmse = 0.056384
Epoch 15/64: val rmse = 0

In [15]:
pd.DataFrame({'id': train.id, TARGET: oof_preds}).to_csv('oof_realmlp_plus_origcol.csv', index=False)
pd.DataFrame({'id': test.id, TARGET: test_preds}).to_csv('test_realmlp_plus_origcol.csv', index=False)